In [6]:
def get_web_data(dom=None, cookies=None):
    '''
    获取每页评论数据
    '''
    # //这里可以表示多个目录，当有多个目录时可以用//来代替，@class="comment-item"表示标签下的特定属性，这里用于定位特定的div标签，text()表示标签的文本信息
    names = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/a/text()')           # 用户名
    ratings = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/span[2]/@class')   # 评分
    times = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/span[@class="comment-time "]/@title')     # 评论发布时间
    message = dom.xpath('//div[@class="comment-item"]//div[@class="comment"]//span[@class="short"]/text()')    # 短评正文
    user_url = dom.xpath('//div[@class="comment-item"]//span[@class="comment-info"]/a/@href')           # 用户主页网址
    votes = dom.xpath('//div[@class="comment-item"]//div[@class="comment"]//span[@class="votes"]/text()')    # 赞同数量
    cities = []
    load_times = []
    for i in user_url:
        web_data = requests.get(i, cookies=cookies)
        dom_url = etree.HTML(web_data.text, etree.HTMLParser(encoding='utf-8'))
        address = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/a/text()')                    # 用户居住地
        load_time = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/div[@class="pl"]/text()')   # 用户入会时间
        cities.append(address)
        load_times.append(load_ time)
        time.sleep(3)
    ratings = ['' if 'rating' not in i else int(re.findall('\d{2}', i)[0]) for i in ratings]   # 评分数据整理
    load_times = ['' if i == [] else i[1].strip()[:-2] for i in load_times]   # 入会数据整理
    cities = ['' if i == [] else i[0] for i in cities]   # 居住地数据整理
    data = pd.DataFrame({
        '用户名': names,
        '居住城市': cities,
        '加入时间': load_times,
        '评分': ratings,
        '发表时间': times,
        '短评正文': message,
        '赞同数量': votes
    })
    return data

In [7]:
from selenium import webdriver
from lxml import etree
import requests
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [8]:
driver = webdriver.Chrome()   # 启动chrome浏览器
url = 'https://movie.douban.com/subject/26266893/comments?status=P'
driver.get(url)   # 获取网页源码数据

In [9]:
# 通过cookies来进行登录操作
cookies_str = 'bid=-KCZGWlMR4I; ap_v=0,6.0; dbcl2="219869902:Xw87UAj00d8"; ck=epfK; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1595579801%2C%22https%3A%2F%2Faccounts.douban.com%2Fpassport%2Fregister%22%5D; _pk_id.100001.8cb4=969a41245f42aff2.1595579801.1.1595579801.1595579801.; _pk_ses.100001.8cb4=*; push_noty_num=0; push_doumail_num=0'
cookies = {}
for i in cookies_str.split(';'):  # 将cookies整理成所需格式，这里整理为字典格式
    k, v = i.split('=', 1)  # 1表示只按照第一个等号进行拆分，后面如果还有等号则不进行拆分
    cookies[k] = v

In [10]:
# 对所有页面进行数据爬取及解析操作，并进行数据保存
all_data = pd.DataFrame()  # 用于将所有清洗好的数据加载到数据框内(每清洗一页数据就加载一夜数据)
wait = WebDriverWait(driver, 20)  # 设置网页加载等待时间，用于实现如果在规定的时间内网页没有加载进来则报错
# 由于要爬取的网站有反爬虫的机制，所以要添加header属性，不添加会返回418
header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"}
while True:
    
    wait.until(  # 这里用网页加载等待时间来判定一个网页是否加载进来
        EC.element_to_be_clickable(   # 通过该项条件确认网页是否已经加载进来,防止断网导致加载不成功
            (By.CSS_SELECTOR, '#comments > div:nth-child(20) > div.comment > h3 > span.comment-info > a') # 当前页最后一个“用户”按钮是否可以点击
        )  # 如果在规定的时间内不能加载网页则终止循环操作
    )
    
    dom = etree.HTML(driver.page_source, etree.HTMLParser(encoding='utf-8'))   # 网页源码解析，得到一个dom文件
    data = get_web_data(dom=dom, cookies=cookies)
    all_data = pd.concat([all_data, data], axis=0)  # 将每一页的获取到的数据进行拼接
    # 在评论的最后一页中下一页按钮是无法点击的，同时在源码中没有selector，所以通过查找来决定循环的跳出
    if driver.find_element_by_css_selector('#paginator > a.next')==[]:   # 判定是否还有“后页”按钮
        break  
    
    confirm_bnt = wait.until(  # 点击按钮是否加载且是否可以被点击
        EC.element_to_be_clickable(
            (By.CSS_SELECTOR, '#paginator > a.next')
        )
    )
    confirm_bnt.click()   # 执行翻页操作
    

AttributeError: 'NoneType' object has no attribute 'xpath'

In [7]:
all_data.to_csv('douban111.csv', index=None, encoding='GB18030')   # 将数据以csv文件的方式写出

In [6]:
all_data.shape

(500, 7)